In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import numpy as np
import wandb
import os
import random
from time import time

from lib.data import EpisodeDataset

In [2]:
class FullyConnected(nn.Module):
    def __init__(self, obs_size=21, obs_width=2, hidden_size=256):
        super().__init__()
        self.input_size = obs_size*obs_width*2

        self.linearNN = nn.Sequential(
              nn.Linear(self.input_size, hidden_size),
              nn.ReLU(),
            #   nn.Linear(hidden_size, hidden_size),
            #   nn.ReLU(),
            #   nn.Linear(hidden_size, hidden_size),
            #   nn.ReLU(),
            #   nn.Linear(hidden_size, hidden_size),
            #   nn.ReLU()
            )
        
        self.action = nn.Sequential(
              nn.Linear(hidden_size, hidden_size),
              nn.ReLU(),
              nn.Linear(hidden_size, 3)
        )

    def forward(self,seq):
        pred = self.linearNN(seq.reshape(-1, self.input_size))

        action = F.log_softmax(self.action(pred), dim=1)
        
        return action

In [3]:
WIDTH = 5
OBS_SIZE = 46
HIDDEN_SIZE = 512

In [4]:
TRAIN_PATH_1 = '/home/bdemoss/Desktop/RL IDM/Episode Data/Random Data/30 TPS/3v3 Train'
TRAIN_PATH_2 = '/home/bdemoss/Desktop/RL IDM/Episode Data/Nexto Data/30 TPS/3v3 Train'

train_dataset = EpisodeDataset([TRAIN_PATH_1, TRAIN_PATH_2], width=WIDTH, include_change=True, corrupt=True)

print(len(train_dataset))

Loading Episode Data
Finished Loading Episode Data
311700


In [5]:
Train_Data_Loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=16)

In [6]:
#model = ConvNet(obs_size=OBS_SIZE, obs_width=WIDTH, conv_number=10, hidden_size=HIDDEN_SIZE)
model = FullyConnected(obs_size=OBS_SIZE, obs_width=WIDTH, hidden_size=HIDDEN_SIZE)

gpumodel = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gpumodel.parameters(), lr=0.0001)

In [5]:
gpumodel.load_state_dict(torch.load('trained_networks/fc3_all_512x4_w_change_corrupt_w_og.pt'))

<All keys matched successfully>

In [6]:
wandb.init(project="Inverse Dynamics Model", entity="harrymead")

wandb.config = {
  "learning_rate": 0.0001,
  "epochs": 10,
  "batch_size": 10
}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: harrymead. Use `wandb login --relogin` to force relogin


In [7]:
TARGET = 0

In [8]:
gpumodel.train()

for epoch in range(10):
    batch = 0
    total_loss = 0
    obs_count = 0
    total_correct = 0
    
    for _obs, _act, _add_data in Train_Data_Loader:
        # TRAINING 
        obs = _obs[0].cuda()
        act = torch.cat((_act[0], _add_data[0][:,0:1]), dim=1).long().cuda()
        add_data = _add_data[0]

        if obs.shape[0] == 0:
            continue

        optimizer.zero_grad()

        y_pred = model(obs)
        pred_act = torch.argmax(y_pred, dim=1)


        loss = criterion(y_pred, act[:, TARGET])

        loss.backward()
        optimizer.step()


        # LOGGING
        obs_count += len(obs)
        total_loss += loss.item() * len(obs)

        correct = act[:, TARGET] == pred_act
        total_correct += torch.sum(correct, dim=0)


        batch += 1
        if batch % 1000 == 0:
            idv_accuracy = total_correct/obs_count

            print(f'Epoch {epoch + 1} - Loss: {total_loss / obs_count}, Correct: {idv_accuracy}')
            # wandb.log({"loss": total_loss/obs_count, "accuracy/throttle": idv_accuracy[0], "accuracy/steer": idv_accuracy[1], "accuracy/yaw": idv_accuracy[2], 
            #             "accuracy/pitch": idv_accuracy[3], "accuracy/roll": idv_accuracy[4], "accuracy/jump": idv_accuracy[5], "accuracy/boost": idv_accuracy[6],
            #             "accuracy/handbrake": idv_accuracy[7], "accuracy/on_ground": idv_accuracy[8]})

            total_loss = 0
            obs_count = 0
            total_correct = 0

Epoch 1 - Loss: 0.6072411048334925, Correct: 0.7321323752403259
Epoch 1 - Loss: 0.4171096659805904, Correct: 0.8365329504013062
Epoch 1 - Loss: 0.3467875951761963, Correct: 0.8728979825973511
Epoch 1 - Loss: 0.3056502761200334, Correct: 0.8908929228782654
Epoch 1 - Loss: 0.28382529862504957, Correct: 0.901045560836792
Epoch 1 - Loss: 0.26493435898601625, Correct: 0.9108210206031799
Epoch 1 - Loss: 0.24457381846709383, Correct: 0.9184700846672058
Epoch 1 - Loss: 0.23058932912726773, Correct: 0.9241077899932861
Epoch 1 - Loss: 0.2249167845145205, Correct: 0.9262722134590149
Epoch 1 - Loss: 0.21473574039917992, Correct: 0.9305453300476074
Epoch 1 - Loss: 0.21778084393104863, Correct: 0.9305371046066284
Epoch 1 - Loss: 0.1996282979432869, Correct: 0.9338614344596863
Epoch 1 - Loss: 0.19461585618716995, Correct: 0.9362353682518005
Epoch 1 - Loss: 0.19523326732974053, Correct: 0.9363150000572205
Epoch 1 - Loss: 0.1867398199871154, Correct: 0.9381107091903687
Epoch 1 - Loss: 0.177301960653628

KeyboardInterrupt: 

In [1]:
torch.save(gpumodel.state_dict(), "trained_networks/single_throttle.pt")

NameError: name 'torch' is not defined

In [9]:
#TEST_PATH = '/home/bdemoss/Desktop/RL IDM/Episode Data/Nexto Data/30 TPS/3v3 Test'
#TEST_PATH = '/home/bdemoss/Desktop/RL IDM/Episode Data/Human Example Data/30 TPS'
TEST_PATH = '/home/bdemoss/Desktop/RL IDM/Episode Data/Human Replays'

test_dataset = EpisodeDataset([TEST_PATH], width=WIDTH, include_change=True, corrupt=False)

Loading Episode Data
Finished Loading Episode Data


In [10]:
Test_Data_Loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

In [11]:
conf_mtx = torch.zeros((3,3)).cuda()
CONF_MAT = True

gpumodel.eval()

total_loss = 0
obs_count = 0
total_correct = 0

correct_on_ground = torch.zeros(9).cuda()
total_on_ground = 0

for _obs, _act, _add_data in Test_Data_Loader:
    # EVALUATION
    obs = _obs[0].cuda()
    act = torch.cat((_act[0], _add_data[0][:,0:1]), dim=1).long().cuda()
    add_data = _add_data[0].long()

    if obs.shape[0] == 0:
        continue

    with torch.no_grad():
        y_pred = model(obs)

    y_pred = model(obs)
    pred_act = torch.argmax(y_pred, dim=1)


    loss = criterion(y_pred, act[:, TARGET])


    # LOGGING
    obs_count += len(obs)
    total_loss += loss.item() * len(obs)

    correct = act[:, TARGET] == pred_act

    total_correct += torch.sum(correct, dim=0)

    if CONF_MAT:
        for i in range(obs.shape[0]):
            conf_mtx[act[i, TARGET], pred_act[i].long()] += 1

    

print(f'Loss: {total_loss / obs_count}, Correct: {total_correct / obs_count}')

np.set_printoptions(precision=4, suppress=True)
print((conf_mtx).cpu().numpy())

Loss: 0.46652787923812866, Correct: 0.8958847522735596
[[ 173.   16.    8.]
 [  28.  804.   50.]
 [   4.  147. 1200.]]


In [12]:
print(pred_act)
print(act[:])

tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0')
